In [1]:
import numpy  as np
import pandas as pd

In [18]:
df = pd.read_excel("../DataSets/cowing.xlsx")
y = df["y"]
x1 = df["X1"]
x2 = df["X2"]
x3 = df["X3"]
p1 = df["P1"]
p2 = df["P2"]
p3 = df["P3"]

0      6.955593
1      7.255591
2      7.351800
3      7.312553
4      6.517671
         ...   
106    5.214936
107    6.111467
108    7.294377
109    8.142645
110    5.267858
Name: y, Length: 111, dtype: float64

In [17]:
def AppLogDen_ALS77(Pars: pd.Series):
    
    global y, x1, x2, x3, p1, p2, p3
    
    [alpha, beta1, beta2, beta3, sigma2u, sigma2v] = Pars[0:6]
    
    Lambda = np.sqrt(sigma2u / sigma2v)
    sigma2 = sigma2u + sigma2v
    sigma  = np.sqrt( sigma2 )
    
    eps = y - alpha - x1*beta1 - x2*beta2 - x3*beta3
    B2  = p2 - p1 + np.log(beta1) - np.log(beta2)
    B3  = p3 - p1 + np.log(beta1) - np.log(beta3)
    w1  = x1 - x2 - B2
    w2  = x1 - x3 - B3
    w   = pd.concat([w1, w2], axis=1)
    print(w)
    #Den = 2/

            0         1
0   -0.185772  2.914729
1    0.203163  3.570886
2    0.243893  3.088265
3    1.150068  4.010421
4    1.279613  3.200047
..        ...       ...
106  0.833264  3.204064
107  0.462516  1.713923
108  0.419967  2.132848
109  0.540051  3.951360
110  1.150923  3.568176

[111 rows x 2 columns]


In [ ]:
def AppLogLikelihood_ALS77(coefs: pd.Series):
    
    #~~~~ transform parametrs back true range ~~~~#
    coefs[4:6] = np.exp(coefs[4:6])
    
    #~~~~ obtain the log likelihood ~~~~#
    logDen = AppLogDen_ALS77(coefs)
    return -sum(logDen)

In [ ]:
def AppEstimate_ALS77():